# XCS224N Natural Language Processing with Deep Learning


# Lecture 2

[CS224N](http://web.stanford.edu/class/cs224n/) / [XCS224N](http://scpd.stanford.edu/search/publicCourseSearchDetails.do?method=load&courseId=93933715) / [Lecture](https://youtu.be/kEMJRjEdNzM) / [Slides](http://web.stanford.edu/class/cs224n/slides/cs224n-2019-lecture02-wordvecs2.pdf)

## More on Word2Vec

Reviewing Word2Vec once again...

<img src="images/word2vec_example.PNG" />

$$P(o|c) = \frac{exp(u_{O}^{T}v_{c})}{\sum_{w\in V}exp(u_{W}^{T}v_{c})}$$

### Word2Vec parameters and computations

In most libraries, such as TensorFlow and PyTorch, word vectors are the rows contrary to columns in linear algebra.

<img src="images/matrices.PNG" />

The output is a probability distribution for all words, giving reasonably high probabilities to words that often occur in the context of a given word.

However, what about <strong>"the"</strong>, <strong>"and"</strong>, and so on. They are very common as context words and will have a high dot product value. This can be referred to as the high frequency probability effect.

### Word2Vec maximizes objective function by putting similar words nearby in space

<img src="images/word_vector_example.PNG" />

There is a drawback to these visualisations though. Words can be close to one another in many different directions (across multiple dimensions). This is where 2 projections can be misleading.

## Optimization Basics

### Gradient Descent

We have a cost function $J(\theta)$ we would like to minimize 

Utilizing <strong>gradient descent</strong> we can minimize $J(\theta)$

The concept is:
* For the current $\theta$, calculate the gradient of $J(\theta)$
* Then take a small step in the direction of negative gradient (i.e. walking down the hill)
* Repeat until optimized